In [1]:
%reload_ext autoreload
%autoreload 2

from qwanta import Tuner, QuantumNetwork, Configuration, Xperiment
from qwanta import QuantumNetwork, Configuration, Xperiment, Tuner
import pandas as pd
import ast
import numpy as np
import networkx as nx
from qwanta.Qubit import PhysicalQubit
import seaborn as sns
import matplotlib.pyplot as plt

### Define configuration and read the network timeline

As of current implementation, the timeline should be format as in csv or excel file to let Xperiment to be able to read it.
However, Xperiment is just a class wrapping that does the validation of the timeline and the network, so you can use any other format as long as you can read it and pass it to the Xperiment class.

In [83]:
def adjusted_rate(rl, dl, ds, loss=0.1):
    return rl*10**((dl - ds)*loss/10)

minimal_edges_infos = [(0.0001, 100, 100), (0.0001, 175, 25), 
                       (0.0001, 150, 50) ,(0.0001, 125, 75)]
adjusted_rate_list = [adjusted_rate(*i) for i in minimal_edges_infos]

adjusted_rate_list

[0.0001, 0.0031622776601683794, 0.001, 0.000316227766016838]

In [6]:
# Hyperparameters

loss = 0.03 # dB/km
depo_prob = 0.025
gate_error = 0.0
measurement_error = 0.0
memory_time = 0.01 # second

num_hops = 2
num_nodes = num_hops + 1

# 3 Equidistant nodes
node_info = {f'Node {i}': {'coordinate': (int(i*100), 0, 0)} for i in range(num_nodes)}

# 3 non-equidistant nodes
# node_info = {'Node 0': {'coordinate': (0, 0, 0)},
#  'Node 1': {'coordinate': (50, 0, 0)},
#  'Node 2': {'coordinate': (200, 0, 0)}}

# default edge info
edge_info = {
    (f'Node {i}', f'Node {i+1}'): {
    'connection-type': 'Space',
    'depolarlizing error': [1 - depo_prob, depo_prob/3, depo_prob/3, depo_prob/3],
    'loss': loss,
    'light speed': 300000,
    'Pulse rate': 0.00001,
    f'Node {i}':{
        'gate error': gate_error,
        'measurement error': measurement_error,
        'memory function': memory_time
    },
    f'Node {i+1}':{
        'gate error': gate_error,
        'measurement error': measurement_error,
        'memory function': memory_time
    },
    }
for i in range(num_hops)}

# edge with adjust pulse rate
# edge_info = {('Node 0', 'Node 1'): 
#         {'connection-type': 'Space',
#         'depolarlizing error': [1, 0.0, 0.0, 0.0],
#         'loss': 0.1,
#         'light speed': 300000,
#         'Pulse rate': adjusted_rate(0.0001, 150, 50, loss=loss),
#         'Node 0': {'gate error': 0.1, 'measurement error': 0, 'memory function': 1},
#         'Node 1': {'gate error': 0.1, 'measurement error': 0, 'memory function': 1}},

#         ('Node 1', 'Node 2'): {'connection-type': 'Space',
#         'depolarlizing error': [1, 0.0, 0.0, 0.0],
#         'loss': 0.1,
#         'light speed': 300000,
#         'Pulse rate': 1e-04,
#         'Node 1': {'gate error': 0.1, 'measurement error': 0, 'memory function': 1},
#         'Node 2': {'gate error': 0.1, 'measurement error': 0, 'memory function': 1}}}

exps = Xperiment(
    timelines_path = 'network/exper_id3_selectedStats_2hops.xlsx',
    nodes_info_exp = node_info,
    edges_info_exp = edge_info,
    gate_error = gate_error,
    measurement_error = measurement_error,
    memory_time = memory_time,
    strategies_list=['0G']
)

### To validate the experiment configuration, and network topology

You can validate the experiment configuration and network topology by running the following command. 
If vis is set to True, the network topology will be visualized using pyvis library, you could see the network topology in the browser using the generated html file.
If get_table is set to True, the network topology will be printed in table format show the summary of validation.
If show_message is set to True, the validation message will be printed.

In [59]:
# exps.validate(get_table=True, vis=True)

### To run the experiment
You could run the experiment by running the following command.
Set save_result to True to save the experiment result to the experiment result folder.

In [7]:
results = exps.execute()

### Let us see the result of 0G experiment
The result dictionary contains the following keys:

In [8]:
all_keys = []
for key in results['0G'].keys():
    # print(key)
    if key != 'Qubits waiting time':
        all_keys.append(key)
    else:
        pass

all_keys

['numPhysicalBuffer',
 'numInternalEncodingBuffer',
 'numInternalDetectingBuffer',
 'numInternalInterfaceBuffer',
 'memory_function',
 'gate_error',
 'measurement_error',
 'timeline',
 'experiment',
 'light_speed_in_fiber',
 'message',
 'g',
 'result_path',
 'label_recorded',
 'collectFidelityHistory',
 'simulation_time',
 'coor_system',
 'nodes_info',
 'NetworkTopology',
 'throughtputEdges',
 'fidelity',
 'Resources Produced',
 'Base Resources Attempt',
 'Resource Used in Fidelity Estimation',
 'Time used',
 'Fidelity Estimation Time',
 'Fidelity History',
 'Commutation inforamtion',
 'Simulation log']

In [9]:
def visualize(result):
    # to_print = ['fidelity', 'simulation_time', 'Resources Produced', 'Base Resources Attempt', 'Resource Used in Fidelity Estimation', 'Time used', 'Fidelity Estimation Time']
    # to_print = ['fidelity', 'Base Resources Attempt', 'Fidelity Estimation Time', 'Time used']
    to_print = all_keys
    for key in to_print:
        print(f'{key}: {result[key]}')


In [11]:
visualize(results['0G'])

numPhysicalBuffer: 20
numInternalEncodingBuffer: 20
numInternalDetectingBuffer: 10
numInternalInterfaceBuffer: 2
memory_function: <function Xperiment.__init__.<locals>.memory_error_function at 0x158cc6c20>
gate_error: 0.0
measurement_error: 0.0
timeline: [{'Main Process': 'Generate physical Bell pair', 'Edges': ['Node 0', 'Node 1'], 'Num Trials': True, 'Label in': 'Physical Qubit', 'Label out': 'Physical', 'Resource Type': nan, 'Protocol': nan, 'Note': nan, 'Schedule Config': None, 'isSuccess': 0}, {'Main Process': 'Generate physical Bell pair', 'Edges': ['Node 1', 'Node 2'], 'Num Trials': True, 'Label in': 'Physical Qubit', 'Label out': 'Physical', 'Resource Type': nan, 'Protocol': nan, 'Note': nan, 'Schedule Config': None, 'isSuccess': 0}, {'Main Process': 'Entanglement swapping', 'Edges': ['Node 0', 'Node 1', 'Node 2'], 'Num Trials': True, 'Label in': 'Physical', 'Label out': 'k', 'Resource Type': 'Physical', 'Protocol': nan, 'Note': nan, 'isSuccess': 0}, {'Main Process': 'State tom

In [102]:
9000/(results['1-2G-DirectedEncoded']['Time used']+results['1-2G-DirectedEncoded']['Fidelity Estimation Time'])

1297.984566482855

In [53]:
data = {
    'Experiment name': [
        'Equidistant nodes', 
        'Non-equidistant nodes', 
        'Non-equidistant nodes, adjust pulse rate (1e-05, 1e-07)',
        'Non-equidistant nodes, adjust pulse rate (1e-03, 1e-09)'],
    'Fidelity': [
        0.8176666666666667, 
        0.7966666666666667, 
        0.7838333333333333,
        0.8043333333333332],
    'Base Resources Attempt': [
        180644, 313271, 312074, 311333],
    'Fidelity Estimation Time': [
        3.5771033333330187, 
        14.684520000006458, 
        14.625501000006386,
        28.466833333345136]
}

df = pd.DataFrame(data)
df

,Experiment name,Fidelity,Base Resources Attempt,Fidelity Estimation Time
0,Equidistant nodes,0.817667,180644,3.577103
1,Non-equidistant nodes,0.796667,313271,14.684520
2,"Non-equidistant nodes, adjust pulse rate (1e-0...",0.783833,312074,14.625501
3,"Non-equidistant nodes, adjust pulse rate (1e-0...",0.804333,311333,28.466833


In [46]:
# print all generation results
for key in results.keys():
    print(f'Generation {key}')
    visualize(results[key])
    print('\n')

Generation 0G
fidelity: 0.8196666666666667
Base Resources Attempt: 178955
Fidelity Estimation Time: 3.5787966666663524


Generation 1G-Ss-Dp
fidelity: 0.6646666666666666
Base Resources Attempt: 693019
Fidelity Estimation Time: 13.95895000001088


Generation 2G-NonLocalCNOT
fidelity: 0.24466666666666667
Base Resources Attempt: 1264704
Fidelity Estimation Time: 29.07743333335874


Generation 1-2G-DirectedEncoded
fidelity: 0.24833333333333332
Base Resources Attempt: 690898
Fidelity Estimation Time: 13.889330000010748


Generation HG-END2ENDPurifiedEncoded
fidelity: 0.4666666666666667
Base Resources Attempt: 11154
Fidelity Estimation Time: 0.24849666666666453


Generation 1-2G-DirectedEncoded-Perfect
fidelity: 0.27599999999999997
Base Resources Attempt: 692104
Fidelity Estimation Time: 13.886480000010769


Generation 2G-NonLocalCNOT-Perfect
fidelity: 0.2495
Base Resources Attempt: 1264243
Fidelity Estimation Time: 29.173070000025593


